# Cycle GAN

The dataset should be prepared following this [guide](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#cyclegan-datasets)

Approaches for converting a layer map from low-res layer map to high-res detailed b-scan:

In [ ]:
import idp_utils.data_handling.constants as C

%cd $C.ROOT_PATH

Here we store some old code snipsets for cycle gan:

They will be removed later.

```python
random.seed(6)

def split_files(file_names, train_ratio, test_ratio):
    ''' split file names into train, val, test and return as a dictionary'''
    random.shuffle(file_names)
    
    num_train = int(len(file_names) * train_ratio)
    num_test = int(len(file_names) * test_ratio)
    
    train_files = file_names[:num_train]
    test_files = file_names[num_train:num_train+num_test]
    val_files = file_names[num_train+num_test:]
    
    splited_files = { 'train': train_files, 'test': test_files, 'val': val_files}
    return splited_files

def create_cycle_dataset(src_folder, dst_folder, group='A', train_ratio=0.85, test_ratio=0.15):
    '''It will travers all files in src folder '''
    src_files = glob(os.path.join(src_folder, '*'))
    splited_files = split_files(src_files, train_ratio, test_ratio)
    for split, files in splited_files.items():
        dst_folder_split = os.path.join(dst_folder, split+group)
        Path(dst_folder_split).mkdir(parents=True, exist_ok=True)
        
        for src_path in files:
            dst_path = os.path.join(dst_folder_split, src_path.split('/')[-1])
            shutil.copy(src_path, dst_path)
```

```python
dst_folder = C.DATASET_PATTERN.format(data='CGAN', name='OP2AROI')
src_folder = C.LAYER_PATTERN.format(data='OP',dtype='original')
create_cycle_dataset(src_folder, dst_folder, group='A')

dst_folder = C.DATASET_PATTERN.format(data='CGAN', name='OP2AROI')
src_foler = C.BSCAN_PATTERN.format(data='AROI',dtype='original')
create_cycle_dataset(src_folder, dst_folder, group='B')
```

```bash
$ nohup python pytorch-CycleGAN-and-pix2pix/train.py --dataroot ./data/datasets/CGAN/OP2AROI --name op2aroi --model cycle_gan
```

# 1 Direct Mapping from Label to Bscan between Domains

In [ ]:
import os
from pathlib import Path
from glob import glob
import random
import shutil

random.seed(6)

def split_files(file_names, train_ratio, test_ratio):
    ''' split file names into train, val, test and return as a dictionary'''
    random.shuffle(file_names)
    
    num_train = int(len(file_names) * train_ratio)
    num_test = int(len(file_names) * test_ratio)
    
    train_files = file_names[:num_train]
    test_files = file_names[num_train:num_train+num_test]
    val_files = file_names[num_train+num_test:]
    
    splited_files = { 'train': train_files, 'test': test_files, 'val': val_files}
    return splited_files

def create_cycle_dataset(src_folder, dst_folder, train_ratio=0.8, test_ratio=0.1):
    '''It will travers all files in src folder '''
    src_files = glob(os.path.join(src_folder, '*'))
    splited_files = split_files(src_files, train_ratio, test_ratio)
    for split, files in splited_files.items():
        dst_folder_split = os.path.join(dst_folder, split)
        Path(dst_folder_split).mkdir(parents=True, exist_ok=True)
        for src_path in files:
            dst_path = os.path.join(dst_folder_split, src_path.split('/')[-1])
            shutil.copyfile(src_path, dst_path)

## Approach 1
1. Convert the low-res layer map to low-res b-scan with pix2pix
2. Convert the low-res b-scan to high-res detailed b-scan with CycleGAN

### Create Datasets

First we need to create a dataset that maps a low-res layer map to low-res b-scan

In [ ]:
label_folder = C.SPLIT_PATTERN.format(data='OP', name='original') + '/labels'
bscan_folder = C.SPLIT_PATTERN.format(data='OP', name='original') + '/bscans'
dataset_folder = C.DATASET_PATTERN.format(data='OP', name='original')
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py --fold_A $label_folder --fold_B $bscan_folder --fold_AB $dataset_folder

Then we create a dataset that maps low-res b-scan to high-res bscan

In [ ]:
# Source domain: OP bscan
src_foler = C.BSCAN_PATTERN.format(data='OP',dtype='original')
dst_folder = C.DATASET_PATTERN.format(data='OP', name='OP_BSCAN_CGAN')
create_cycle_dataset(src_foler, dst_folder)

In [ ]:
# Target domain: AROI bscan
src_foler = C.BSCAN_PATTERN.format(data='AROI')
dst_folder = C.DATASET_PATTERN.format(data='AROI', name='AROI_BSCAN_CGAN')
create_cycle_dataset(src_foler, dst_folder)

In [ ]:
# Manually copy folers to trainA and trainB
dataset_root = C.DATASET_PATTERN.format(data='CGAN', name='OPBSCAN2AROI')
dst_trainA = os.path.join(dataset_root, 'trainA')
dst_trainB = os.path.join(dataset_root, 'trainB')
dst_testA = os.path.join(dataset_root, 'testA')
dst_testB = os.path.join(dataset_root, 'testB')
src_trainA = os.path.join(C.DATASET_PATTERN.format(data='OP', name='OP_BSCAN_CGAN'), 'train', '*')
src_trainB = os.path.join(C.DATASET_PATTERN.format(data='AROI', name='AROI_BSCAN_CGAN'), 'train', '*')
src_testA = os.path.join(C.DATASET_PATTERN.format(data='OP', name='OP_BSCAN_CGAN'), 'test', '*')
src_testB = os.path.join(C.DATASET_PATTERN.format(data='AROI', name='AROI_BSCAN_CGAN'), 'test', '*')
Path(dst_trainA).mkdir(parents=True, exist_ok=True)
Path(dst_trainB).mkdir(parents=True, exist_ok=True)
Path(dst_testA).mkdir(parents=True, exist_ok=True)
Path(dst_testB).mkdir(parents=True, exist_ok=True)

# If follow_symlinks is false, and src is a symbolic link, dst will be created as a symbolic link.
for src, dst in zip([src_trainA, src_trainB, src_testA, src_testB], [dst_trainA, dst_trainB, dst_testA, dst_testB]):
    for src_f in glob(src):
            shutil.move(src_f, dst)

### Train a map from low-res B-scan to high-res B-scan (cycle GAN)

In [ ]:
checkpoint_name = 'opbscan2aroi'

!NVIDIA_VISIBLE_DEVICES=1 python pytorch-CycleGAN-and-pix2pix/train.py --dataroot $dataset_root \
    --name $checkpoint_name \
    --model cycle_gan

### Train a mapping from low-res layer map to low-res B-scan (pix2pix)

In [ ]:
dataset_folder = C.DATASET_PATTERN.format(data='OP', name='original')
checkpoint_name = 'op_original_pix2pix'

!NVIDIA_VISIBLE_DEVICES=1 python pytorch-CycleGAN-and-pix2pix/train.py \
        --dataroot $dataset_folder \
        --name $checkpoint_name \
        --model pix2pix \
        --direction AtoB \
        --n_epochs 100 \
        --print_freq 500 \
        --batch_size 4

### Cnvert low-res layer maps to high-res bscans in 2 steps

Stage 1, we convert low-res layer maps to low-res bscans with pix2pix

In [ ]:
dataset_folder = C.DATASET_PATTERN.format(data='OP', name='original')
checkpoint_name = 'op_original_pix2pix'
results_dir = './results/opl2aroib'
!python pytorch-CycleGAN-and-pix2pix/test.py --dataroot $dataset_folder --direction AtoB --model pix2pix --name $checkpoint_name --results_dir $results_dir

Copy the generated (fake) low-res to another folder for the next stage

In [ ]:
import shutil
from glob import glob
from pathlib import Path
checkpoint_name = 'op_original_pix2pix'
src_root = os.path.join(results_dir, checkpoint_name, 'test_latest', 'images')
stage1_fake_bscan_dir = os.path.join(results_dir, 'fake_op_bscan')
stage1_fake_bscan_dir_testa = os.path.join(stage1_fake_bscan_dir, 'testA')
Path(stage1_fake_bscan_dir_testa).mkdir(exist_ok=True, parents=True)
fake_bscan_pattern = os.path.join(src_root, '*_fake_B.png')
fake_bscan_paths = glob(fake_bscan_pattern)
for fake_bscan in fake_bscan_paths:
    shutil.copy(fake_bscan, stage1_fake_bscan_dir_testa)

Stage 2: convert fake low-res bscans to high-res bscans:

We copy test B from OPBSCAN2AROI dataset

In [ ]:
opbscan2aroi_dataset_root = C.DATASET_PATTERN.format(data='CGAN', name='OPBSCAN2AROI')
shutil.copytree(os.path.join(opbscan2aroi_dataset_root, 'testB'), os.path.join(stage1_fake_bscan_dir, 'testB'))

In [ ]:
checkpoint_name = 'opbscan2aroi'
!python pytorch-CycleGAN-and-pix2pix/test.py --dataroot $stage1_fake_bscan_dir --name $checkpoint_name --model cycle_gan --no_dropout --results_dir $results_dir

Copy generated to a seperate folder for easier recognition

In [ ]:
import shutil
from glob import glob
from pathlib import Path
checkpoint_name = 'opbscan2aroi'
stage2_fake_bscan_dir = os.path.join(results_dir, 'fake_aroi_bscan')
src_root = os.path.join(results_dir, checkpoint_name, 'test_latest', 'images')
Path(stage2_fake_bscan_dir).mkdir(exist_ok=True)
fake_bscan_pattern = os.path.join(src_root, '*_fake_B.png')
fake_bscan_paths = glob(fake_bscan_pattern)
for fake_bscan in fake_bscan_paths:
    shutil.copy(fake_bscan, stage2_fake_bscan_dir)

## Approach 2: Direct translation
Directly translate low-res layer map to high-res B-scan

### Create Datasets:
The datasets for CycleGAN is rather straight forward. We only need 2 folders contraining all pictures in each domain.

We do this by creating soft link to extracted files, and then place the soft links in another dataset folder.

In [ ]:
# Source domain: OP layer map
src_foler = C.LAYER_PATTERN.format(data='OP',dtype='original')
dst_folder = C.DATASET_PATTERN.format(data='OP', name='OP_LAYER_CGAN')
create_cycle_dataset(src_foler, dst_folder)

In [ ]:
# Target domain: AROI bscan
src_foler = C.BSCAN_PATTERN.format(data='AROI')
dst_folder = C.DATASET_PATTERN.format(data='AROI', name='AROI_BSCAN_CGAN')
create_cycle_dataset(src_foler, dst_folder)

In [ ]:
# Manually copy folers to trainA and trainB
dataset_root = C.DATASET_PATTERN.format(data='CGAN', name='OP2AROI')
dst_trainA = os.path.join(dataset_root, 'trainA')
dst_trainB = os.path.join(dataset_root, 'trainB')
dst_testA = os.path.join(dataset_root, 'testA')
dst_testB = os.path.join(dataset_root, 'testB')
src_trainA = os.path.join(C.DATASET_PATTERN.format(data='OP', name='OP_LAYER_CGAN'), 'train', '*')
src_trainB = os.path.join(C.DATASET_PATTERN.format(data='AROI', name='AROI_BSCAN_CGAN'), 'train', '*')
src_testA = os.path.join(C.DATASET_PATTERN.format(data='OP', name='OP_LAYER_CGAN'), 'test', '*')
src_testB = os.path.join(C.DATASET_PATTERN.format(data='AROI', name='AROI_BSCAN_CGAN'), 'test', '*')
Path(dst_trainA).mkdir(parents=True, exist_ok=True)
Path(dst_trainB).mkdir(parents=True, exist_ok=True)
Path(dst_testA).mkdir(parents=True, exist_ok=True)
Path(dst_testB).mkdir(parents=True, exist_ok=True)

# If follow_symlinks is false, and src is a symbolic link, dst will be created as a symbolic link.
for src, dst in zip([src_trainA, src_trainB, src_testA, src_testB], [dst_trainA, dst_trainB, dst_testA, dst_testB]):
    for src_f in glob(src):
            shutil.move(src_f, dst)

In [ ]:
!python pytorch-CycleGAN-and-pix2pix/train.py --dataroot $dataset_root \
    --name op2aroi \
    --model cycle_gan

# Map from Label to Bscan within the Same Domain

### Create Datasets

#### OP Layermap to B-scan

These data can be directly accquired from split folder. We create *hard links* to files in split folder

A no longer needed function:
```python
def create_cycle_dataset_folder(dataset_name):
    dataset_root = C.DATASET_PATTERN.format(data='CGAN', name=dataset_name)
    dst_trainA = os.path.join(dataset_root, 'trainA')
    dst_trainB = os.path.join(dataset_root, 'trainB')
    dst_testA = os.path.join(dataset_root, 'testA')
    dst_testB = os.path.join(dataset_root, 'testB')
    Path(dst_trainA).mkdir(parents=True, exist_ok=True)
    Path(dst_trainB).mkdir(parents=True, exist_ok=True)
    Path(dst_testA).mkdir(parents=True, exist_ok=True)
    Path(dst_testB).mkdir(parents=True, exist_ok=True)

create_cycle_dataset_folder('OP_LAYER2BSCAN')
```

In [ ]:
from pathlib import Path
from glob import glob
import os
import shutil

dataset_root = C.DATASET_PATTERN.format(data='CGAN', name='OP_LAYER2BSCAN')
label_folder = C.SPLIT_PATTERN.format(data='OP', name='original') + '/labels'
bscan_folder = C.SPLIT_PATTERN.format(data='OP', name='original') + '/bscans'

dst_trainA = os.path.join(dataset_root, 'trainA')
dst_trainB = os.path.join(dataset_root, 'trainB')
dst_testA = os.path.join(dataset_root, 'testA')
dst_testB = os.path.join(dataset_root, 'testB')
src_trainA = os.path.join(label_folder, 'train')
src_trainB = os.path.join(bscan_folder, 'train')
src_testA = os.path.join(label_folder, 'test')
src_testB = os.path.join(bscan_folder, 'test')

for src, dst in zip([src_trainA, src_trainB, src_testA, src_testB], [dst_trainA, dst_trainB, dst_testA, dst_testB]):
    shutil.copytree(src, dst, copy_function=os.link, dirs_exist_ok=True)

train it

In [ ]:
python pytorch-CycleGAN-and-pix2pix/train.py --dataroot data/datasets/CGAN/OP_LAYER2BSCAN \
    --name op_layer2bscan \
    --model cycle_gan

Test

In [ ]:
!python pytorch-CycleGAN-and-pix2pix/test.py \
    --dataroot data/datasets/CGAN/OP_LAYER2BSCAN \
    --direction AtoB \
    --name op_layer2bscan \
    --model cycle_gan \
    --results_dir /home/extra/micheal/IDP/results/op_cgan_layer2bscan

#### AROI Layermap to B-scan

In [ ]:
from pathlib import Path
from glob import glob
import os
import shutil

dataset_root = C.DATASET_PATTERN.format(data='CGAN', name='AROI_LAYER2BSCAN')
label_folder = C.SPLIT_PATTERN.format(data='AROI', name='original') + '/labels'
bscan_folder = C.SPLIT_PATTERN.format(data='AROI', name='original') + '/bscans'

dst_trainA = os.path.join(dataset_root, 'trainA')
dst_trainB = os.path.join(dataset_root, 'trainB')
dst_testA = os.path.join(dataset_root, 'testA')
dst_testB = os.path.join(dataset_root, 'testB')
src_trainA = os.path.join(label_folder, 'train')
src_trainB = os.path.join(bscan_folder, 'train')
src_testA = os.path.join(label_folder, 'test')
src_testB = os.path.join(bscan_folder, 'test')

for src, dst in zip([src_trainA, src_trainB, src_testA, src_testB], [dst_trainA, dst_trainB, dst_testA, dst_testB]):
    shutil.copytree(src, dst, copy_function=os.link, dirs_exist_ok=True)

train

In [ ]:
python pytorch-CycleGAN-and-pix2pix/train.py --dataroot data/datasets/CGAN/AROI_LAYER2BSCAN \
    --name aroi_layer2bscan \
    --model cycle_gan

test

In [ ]:
!python pytorch-CycleGAN-and-pix2pix/test.py \
    --dataroot data/datasets/CGAN/AROI_LAYER2BSCAN \
    --direction AtoB \
    --name aroi_layer2bscan \
    --model cycle_gan \
    --results_dir /home/extra/micheal/IDP/results/aroi_cgan_layer2bscan \
    --num_test 377